In [0]:
!pip install pandas

In [26]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

from google.colab import drive
import numpy as np
import cv2
import re
from pathlib import Path
from keras.utils import np_utils
from keras import Sequential
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import AveragePooling2D
from keras.layers import Dropout
from keras.optimizers import *
from keras.applications.vgg16 import VGG16
import pickle
from keras.preprocessing import image
import gc 
import gzip
#%matplotlib inline # only if running in jupyter notebook
import matplotlib
import matplotlib.pyplot as plt
from keras.utils import np_utils

drive.mount('/content/drive')

raw_data_csv_file_name = '/content/drive/My Drive/Colab Notebooks/fer2013.csv'
raw_data = pd.read_csv(raw_data_csv_file_name)
raw_data.info()
raw_data.head()

#img = raw_data["pixels"][0] # first image
#val = img.split(" ")
#x_pixels = np.array(val, 'float32')
#x_pixels /= 255
#x_reshaped = x_pixels.reshape(48,48)
x_train_feature_map , y_train = list(), list()
for i in range(35887):
  img = raw_data["pixels"][i] # first image
  val = img.split(" ")
  x_pixels = np.array(val, 'float32')
  x_pixels /= 255
  #x_reshaped = x_pixels.reshape(48,48)
  x_train_feature_map.append(x_pixels)
#x_train_feature_map.append( raw_data["pixels"][:35887] )
y_train.append( raw_data["emotion"][:35887])
x_train_feature_map = np.array( x_train_feature_map )
y_train = np_utils.to_categorical(y_train)
'''x_train_feature_map = raw_data["pixels"][:35887]
x_train_feature_map = np.array( x_train_feature_map ,'float32')
x_train_feature_map /= 255
x_train_feature_map = x_train_feature_map.reshape(48,48)
y_train = raw_data["emotion"][:35887] '''

#plt.imshow(x_reshaped, cmap= "gray", interpolation="nearest")   
#plt.axis("off")
#plt.show()

vgg16 = VGG16(include_top=False, input_shape=(48, 48, 3), weights='imagenet')
topLayerModel = Sequential()
topLayerModel.add(Dense(256, input_shape=(512,), activation='relu'))
topLayerModel.add(Dense(256, input_shape=(256,), activation='relu'))
topLayerModel.add(Dropout(0.5))
topLayerModel.add(Dense(128, input_shape=(256,), activation='relu'))
topLayerModel.add(Dense(7, activation='softmax'))

topLayerModel.summary()
adamax = Adamax()

topLayerModel.compile(loss='categorical_crossentropy', optimizer=adamax, metrics=['accuracy'])

topLayerModel.fit(x_train_feature_map, y_train, validation_data=(x_train_feature_map, y_train), epochs=5, batch_size=1)

inputs = Input(shape=(48, 48, 3))
vg_output = vgg16(inputs)
model_predictions = topLayerModel(vg_output)
final_model = Model(input=inputs, output=model_predictions)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
emotion    35887 non-null int64
pixels     35887 non-null object
Usage      35887 non-null object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_55 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_56 (Dense)             (None, 256)               65792     
_________________________________________________________________
dropout_15 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_57 (Dense)             (None, 128)               32896     
___________________

ValueError: ignored

https://medium.com/@jsflo.dev/training-a-tensorflow-model-to-recognize-emotions-a20c3bcd6468